In [1]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "mH5ygYhtO8NfLRnO7EmPDcbfFIaFS0F5"
os.environ['GRADIENT_WORKSPACE_ID'] = "79d95ad4-f2b3-4c67-a2a7-9f83533964de_workspace"

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('train.csv')
df

,instruction,input,output,text
0,What is the FY2018 capital expenditure amount ...,Table of Contents \n3M Company and Subsidiarie...,$1577.00,You are a financial chatbot trained to answer ...
1,Assume that you are a public equities analyst....,Table of Contents \n3M Company and Subsidiarie...,$8.70,You are a financial chatbot trained to answer ...
2,Is 3M a capital-intensive business based on FY...,3M Company and Subsidiaries\n Consolidated Sta...,"No, the company is managing its CAPEX and Fixe...",You are a financial chatbot trained to answer ...
3,What drove operating margin change as of FY202...,"SG&A, measured as a percent of sales, increase...",Operating Margin for 3M in FY2022 has decrease...,You are a financial chatbot trained to answer ...
4,"If we exclude the impact of M&A, which segment...",Worldwide Sales Change\nBy Business Segment Or...,The consumer segment shrunk by 0.9% organically.,You are a financial chatbot trained to answer ...
...,...,...,...,...
145,Is Verizon a capital intensive business based ...,Consolidated Balance Sheets \nVerizon Communic...,Yes. Verizon's capital intensity ratio was app...,You are a financial chatbot trained to answer ...
146,Has Verizon increased its debt on balance shee...,"At December 31, Maturities \nInterest \nRates ...",No. Verizon's debt decreased by $229 million.,You are a financial chatbot trained to answer ...
147,What is FY2018 days payable outstanding (DPO) ...,Walmart Inc.\nConsolidated Statements of Incom...,42.69,You are a financial chatbot trained to answer ...
148,Based on the information provided primarily in...,Walmart�Inc.\nConsolidated�Statements�of�Incom...,0.2%,You are a financial chatbot trained to answer ...


In [8]:
df['text'].iloc[0]

'You are a financial chatbot trained to answer questions based on the information provided in 10-K documents. Your responses should be directly sourced from the content of these documents. When asked a question, ensure that your answer is explicitly supported by the text in the 10-K filing, and do not include any external information, interpretations, or assumptions not clearly stated in the document. If a question pertains to financial data or analysis that is not explicitly covered in the 10-K filing provided, respond by stating that the information is not available in the document. Your primary focus should be on accuracy, specificity, and adherence to the information in 10-K documents, particularly regarding financial statements, company performance, and market positions.\n### Instruction:\nWhat is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.\n### Input: \nTable of Content

In [13]:
def format_data(df):
    samples = []
    for _,row in df.iterrows():
        input_data = str(row['input'])
        instruction = str(row["instruction"])
        response = str(row['output'])
        samples.append({"inputs": f"### Instruction: {instruction} ### Input: {input_data} \n\n### Response: {response}"})
    return samples

# Applying the function

In [14]:
samples = format_data(df)
samples

[{'inputs': '### Instruction: What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement. ### Input: Table of Contents \n3M Company and Subsidiaries\nConsolidated Statement of Cash Flow s\nYears ended December 31\n \n(Millions)\n    \n2018\n    \n2017\n    \n2016\n \nCash Flows from Operating Activities\n \n \n  \n \n  \n \n  \nNet income including noncontrolling interest\n \n$\n5,363  \n$\n4,869  \n$\n5,058  \nAdjustments to reconcile net income including noncontrolling interest to net cash\nprovided by operating activities\n \n \n  \n \n  \n \n  \nDepreciation and amortization\n \n \n1,488  \n \n1,544  \n \n1,474  \nCompany pension and postretirement contributions\n \n \n(370) \n \n(967) \n \n(383) \nCompany pension and postretirement expense\n \n \n410  \n \n334  \n \n250  \nStock-based compensation expense\n \n \n302  \n \n324  \n \n298  \nGain on sale of businesses\n \n \n(545) 

In [6]:
samples

[{'inputs': '### Instruction: You are a financial chatbot trained to answer questions based on the information provided in 10-K documents. Your responses should be directly sourced from the content of these documents. When asked a question, ensure that your answer is explicitly supported by the text in the 10-K filing, and do not include any external information, interpretations, or assumptions not clearly stated in the document. If a question pertains to financial data or analysis that is not explicitly covered in the 10-K filing provided, respond by stating that the information is not available in the document. Your primary focus should be on accuracy, specificity, and adherence to the information in 10-K documents, particularly regarding financial statements, company performance, and market positions. \n\n### Response: ### Instruction:'},
 {'inputs': '### Instruction: You are a financial chatbot trained to answer questions based on the information provided in 10-K documents. Your re

In [16]:
from gradientai import Gradient

def main():
  with Gradient() as gradient:
      base_model = gradient.get_base_model(base_model_slug="llama2-7b-chat")

      new_model_adapter = base_model.create_model_adapter(
          name="benchmarking_10_labels"
      )
    
      print(f"Created model adapter with id {new_model_adapter.id}")
    

      # this is where fine-tuning happens
      # num_epochs is the number of times you fine-tune the model
      # more epochs tends to get better results, but you also run the risk of "overfitting"
      # play around with this number to find what works best for you
      num_epochs = 10
      count = 0
      while count < num_epochs:
          print(f"Fine-tuning the model, iteration {count + 1}")
          new_model_adapter.fine_tune(samples=samples[:10])
          count = count + 1

      # after fine-tuning
      completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
      print(f"Generated (after fine-tune): {completion}")

      new_model_adapter.delete()

if __name__ == "__main__":
    main()

Created model adapter with id 80243745-8c3b-4c84-a1e6-c5821ddf883a_model_adapter
Fine-tuning the model, iteration 1


ApiException: (422)
Reason: unknown
HTTP response headers: HTTPHeaderDict({'strict-transport-security': 'max-age=63072000; includeSubDomains; preload', 'referrer-policy': 'no-referrer', 'x-content-type-options': 'nosniff', 'x-download-options': 'noopen', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'content-type': 'application/json; charset=utf-8', 'etag': '"13nddtqb23p52"', 'vary': 'Accept-Encoding', 'X-Cloud-Trace-Context': '69bf5d3e1d83c994ac8865832ae40884', 'Date': 'Tue, 02 Jan 2024 04:34:55 GMT', 'Server': 'Google Frontend', 'Content-Length': '182', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"message":"validation: Input validation error: `inputs` tokens + `max_new_tokens` must be <= 2049. Given: 2091 `inputs` tokens and 1 `max_new_tokens`","type":"unprocessableContent"}
